# Notebook for loading wells in a dict of dataframes
### No features calculated yet
### Using knn notebook results to limit to subset of well that have decent neighbors
### Excludes a small list of wells that are difficult to load with LASIO

Last Edits: 2018-07-07 Justin Gosses

In [24]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask import delayed
from dask import compute
# import pdvega
# import vega
import dask.dataframe as dd
from dask.distributed import Client
welly.__version__

'0.3.5'

In [3]:
print(dask.__version__)
print(pd.__version__)

0.17.5
0.23.0


In [4]:
%%timeit
import os
env = %env


70.8 µs ± 972 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
from IPython.display import display

In [6]:
#### Test results Part 1
#### Had to change display options to get this to print in full!
# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [7]:
knn_dir = "../WellsKNN/"

## We're going to load a pickle file of a previously created dataframe

### The pickle is a dataframe that merges:
1. picks_dic = pd.read_csv('../../SPE_006_originalData/OilSandsDB/PICKS_DIC.TXT',delimiter='\t')
2. picks = pd.read_csv('../../SPE_006_originalData/OilSandsDB/PICKS.TXT',delimiter='\t')
3. wells = pd.read_csv('../../SPE_006_originalData/OilSandsDB/WELLS.TXT',delimiter='\t')
4. gis = pd.read_csv('../../well_lat_lng.csv')

### It also excludes any wells that have nulls or zeros for Top McMurray or Base McMurray picks
This was done in notebooks: 
1. notebooks_2018/mapmaking/Map_Exploration_v2-KDtree.ipynb
2. notebooks_2018/Test_RUN_2018_02/DataCleaningPrepof_KNN_neighborPickDepth_df_creation_vA_20180210

### Let's load in the pickle file of the dataframe from previous notebook mentioned above

In [8]:
wells_df_new_cleaned_plus_nn_wNoNulls =  pd.read_pickle(knn_dir+'/'+'wells_df__NB_KDtreePost062018_vA__NoMcTopLeak_v2.p')

In [9]:
wells_df_new_cleaned_plus_nn_wNoNulls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1926 entries, 0 to 2192
Data columns (total 15 columns):
SitID                           1926 non-null int64
McMurray_Base_HorID             1926 non-null int64
McMurray_Top_HorID              1926 non-null int64
McMurray_Base_DEPTH             1926 non-null float64
McMurray_Top_DEPTH              1926 non-null float64
McMurray_Base_Qual              1926 non-null int64
McMurray_Top_Qual               1926 non-null int64
lat                             1926 non-null float64
lng                             1926 non-null float64
UWI                             1926 non-null object
Neighbors_Obj                   1926 non-null object
NN1_McMurray_Top_DEPTH          1921 non-null float64
NN1_McMurray_Base_DEPTH         1926 non-null float64
NN1_thickness                   1921 non-null float64
MM_Top_Depth_predBy_NN1thick    1921 non-null float64
dtypes: float64(8), int64(5), object(2)
memory usage: 240.8+ KB


In [10]:
wells_df_new_cleaned_plus_nn_wNoNulls.head()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI,Neighbors_Obj,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick
0,102496,14000,13000,561.0,475.0,1,3,54.785907,-110.129320,00/12-08-067-01W4/0,"[{'neighbor': 1, 'UWI': '00/09-11-067-02W4/0', 'distance': 0.05753100195547899}, {'neighbor': 2, 'UWI': '00/11-08-068-01W4/0', 'distance': 0.08749430324884858}, {'neighbor': 3, 'UWI': '00/09-11-068-02W4/0', 'distance': 0.10451669032744222}, {'neighbor': 4, 'UWI': '00/06-34-067-02W4/0', 'distance': 0.1099989647224123}, {'neighbor': 5, 'UWI': '00/07-08-067-02W4/0', 'distance': 0.14017282905400244}, {'neighbor': 6, 'UWI': '00/06-26-068-02W4/0', 'distance': 0.14539497462086076}, {'neighbor': 7, 'UWI': '00/10-29-067-02W4/0', 'distance': 0.14676425482044453}]",544.0,630.0,86.0,475.0
1,102497,14000,13000,604.5,515.0,1,3,54.782284,-110.269446,00/07-08-067-02W4/0,"[{'neighbor': 1, 'UWI': '00/10-29-067-02W4/0', 'distance': 0.04734000608365529}, {'neighbor': 2, 'UWI': '00/06-34-067-02W4/0', 'distance': 0.07332213830078355}, {'neighbor': 3, 'UWI': '00/07-36-067-03W4/0', 'distance': 0.07725585850924727}, {'neighbor': 4, 'UWI': '00/10-14-067-03W4/0', 'distance': 0.07846755926496476}, {'neighbor': 5, 'UWI': '00/09-11-067-02W4/0', 'distance': 0.08267376663125782}, {'neighbor': 6, 'UWI': '00/10-08-068-02W4/0', 'distance': 0.09089701215112062}, {'neighbor': 7, 'UWI': '00/06-13-068-03W4/0', 'distance': 0.11681159403073145}]",529.0,613.0,84.0,520.5
2,102498,14000,13000,564.0,480.0,1,3,54.785892,-110.186851,00/09-11-067-02W4/0,"[{'neighbor': 1, 'UWI': '00/12-08-067-01W4/0', 'distance': 0.05753100195547899}, {'neighbor': 2, 'UWI': '00/06-34-067-02W4/0', 'distance': 0.06649378619088428}, {'neighbor': 3, 'UWI': '00/07-08-067-02W4/0', 'distance': 0.08267376663125782}, {'neighbor': 4, 'UWI': '00/09-11-068-02W4/0', 'distance': 0.08728400165551212}, {'neighbor': 5, 'UWI': '00/10-29-067-02W4/0', 'distance': 0.09343691917544036}, {'neighbor': 6, 'UWI': '00/11-08-068-01W4/0', 'distance': 0.10808166919974113}, {'neighbor': 7, 'UWI': '00/10-08-068-02W4/0', 'distance': 0.12013967631468725}]",529.0,613.0,84.0,480.0
3,102500,14000,13000,636.5,549.0,1,3,54.829624,-110.269422,00/10-29-067-02W4/0,"[{'neighbor': 1, 'UWI': '00/10-08-068-02W4/0', 'distance': 0.04355700607250212}, {'neighbor': 2, 'UWI': '00/06-34-067-02W4/0', 'distance': 0.04589036401030715}, {'neighbor': 3, 'UWI': '00/07-08-067-02W4/0', 'distance': 0.04734000608365529}, {'neighbor': 4, 'UWI': '00/07-36-067-03W4/0', 'distance': 0.052016152587450945}, {'neighbor': 5, 'UWI': '00/06-13-068-03W4/0', 'distance': 0.07898840474399109}, {'neighbor': 6, 'UWI': '00/10-14-067-03W4/0', 'distance': 0.08169505420160347}, {'neighbor': 7, 'UWI': '00/11-29-068-02W4/0', 'distance': 0.08752301126560716}]",529.0,613.0,84.0,552.5
4,102501,14000,13000,613.0,529.0,1,2,54.840471,-110.224832,00/06-34-067-02W4/0,"[{'neighbor': 1, 'UWI': '00/10-29-067-02W4/0', 'distance': 0.04589036401030715}, {'neighbor': 2, 'UWI': '00/09-11-068-02W4/0', 'distance': 0.050134469469623355}, {'neighbor': 3, 'UWI': '00/10-08-068-02W4/0', 'distance': 0.055282561346227096}, {'neighbor': 4, 'UWI': '00/09-11-067-02W4/0', 'distance': 0.06649378619088428}, {'neighbor': 5, 'UWI': '00/07-08-067-02W4/0', 'distance': 0.07332213830078355}, {'neighbor': 6, 'UWI': '00/06-26-068-02W4/0', 'distance': 0.07715524006184091}, {'neighbor': 7, 'UWI': '00/11-29-068-02W4/0', 'distance': 0.09180352126144718}]",514.0,603.0,89.0,524.0


In [11]:
print(len(wells_df_new_cleaned_plus_nn_wNoNulls))

1926


In [12]:
wells_df_new_cleaned_plus_nn_wNoNulls.head()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI,Neighbors_Obj,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick
0,102496,14000,13000,561.0,475.0,1,3,54.785907,-110.129320,00/12-08-067-01W4/0,"[{'neighbor': 1, 'UWI': '00/09-11-067-02W4/0', 'distance': 0.05753100195547899}, {'neighbor': 2, 'UWI': '00/11-08-068-01W4/0', 'distance': 0.08749430324884858}, {'neighbor': 3, 'UWI': '00/09-11-068-02W4/0', 'distance': 0.10451669032744222}, {'neighbor': 4, 'UWI': '00/06-34-067-02W4/0', 'distance': 0.1099989647224123}, {'neighbor': 5, 'UWI': '00/07-08-067-02W4/0', 'distance': 0.14017282905400244}, {'neighbor': 6, 'UWI': '00/06-26-068-02W4/0', 'distance': 0.14539497462086076}, {'neighbor': 7, 'UWI': '00/10-29-067-02W4/0', 'distance': 0.14676425482044453}]",544.0,630.0,86.0,475.0
1,102497,14000,13000,604.5,515.0,1,3,54.782284,-110.269446,00/07-08-067-02W4/0,"[{'neighbor': 1, 'UWI': '00/10-29-067-02W4/0', 'distance': 0.04734000608365529}, {'neighbor': 2, 'UWI': '00/06-34-067-02W4/0', 'distance': 0.07332213830078355}, {'neighbor': 3, 'UWI': '00/07-36-067-03W4/0', 'distance': 0.07725585850924727}, {'neighbor': 4, 'UWI': '00/10-14-067-03W4/0', 'distance': 0.07846755926496476}, {'neighbor': 5, 'UWI': '00/09-11-067-02W4/0', 'distance': 0.08267376663125782}, {'neighbor': 6, 'UWI': '00/10-08-068-02W4/0', 'distance': 0.09089701215112062}, {'neighbor': 7, 'UWI': '00/06-13-068-03W4/0', 'distance': 0.11681159403073145}]",529.0,613.0,84.0,520.5
2,102498,14000,13000,564.0,480.0,1,3,54.785892,-110.186851,00/09-11-067-02W4/0,"[{'neighbor': 1, 'UWI': '00/12-08-067-01W4/0', 'distance': 0.05753100195547899}, {'neighbor': 2, 'UWI': '00/06-34-067-02W4/0', 'distance': 0.06649378619088428}, {'neighbor': 3, 'UWI': '00/07-08-067-02W4/0', 'distance': 0.08267376663125782}, {'neighbor': 4, 'UWI': '00/09-11-068-02W4/0', 'distance': 0.08728400165551212}, {'neighbor': 5, 'UWI': '00/10-29-067-02W4/0', 'distance': 0.09343691917544036}, {'neighbor': 6, 'UWI': '00/11-08-068-01W4/0', 'distance': 0.10808166919974113}, {'neighbor': 7, 'UWI': '00/10-08-068-02W4/0', 'distance': 0.12013967631468725}]",529.0,613.0,84.0,480.0
3,102500,14000,13000,636.5,549.0,1,3,54.829624,-110.269422,00/10-29-067-02W4/0,"[{'neighbor': 1, 'UWI': '00/10-08-068-02W4/0', 'distance': 0.04355700607250212}, {'neighbor': 2, 'UWI': '00/06-34-067-02W4/0', 'distance': 0.04589036401030715}, {'neighbor': 3, 'UWI': '00/07-08-067-02W4/0', 'distance': 0.04734000608365529}, {'neighbor': 4, 'UWI': '00/07-36-067-03W4/0', 'distance': 0.052016152587450945}, {'neighbor': 5, 'UWI': '00/06-13-068-03W4/0', 'distance': 0.07898840474399109}, {'neighbor': 6, 'UWI': '00/10-14-067-03W4/0', 'distance': 0.08169505420160347}, {'neighbor': 7, 'UWI': '00/11-29-068-02W4/0', 'distance': 0.08752301126560716}]",529.0,613.0,84.0,552.5
4,102501,14000,13000,613.0,529.0,1,2,54.840471,-110.224832,00/06-34-067-02W4/0,"[{'neighbor': 1, 'UWI': '00/10-29-067-02W4/0', 'distance': 0.04589036401030715}, {'neighbor': 2, 'UWI': '00/09-11-068-02W4/0', 'distance': 0.050134469469623355}, {'neighbor': 3, 'UWI': '00/10-08-068-02W4/0', 'distance': 0.055282561346227096}, {'neighbor': 4, 'UWI': '00/09-11-067-02W4/0', 'distance': 0.06649378619088428}, {'neighbor': 5, 'UWI': '00/07-08-067-02W4/0', 'distance': 0.07332213830078355}, {'neighbor': 6, 'UWI': '00/06-26-068-02W4/0', 'distance': 0.07715524006184091}, {'neighbor': 7, 'UWI': '00/11-29-068-02W4/0', 'distance': 0.09180352126144718}]",514.0,603.0,89.0,524.0


In [19]:
df_new = wells_df_new_cleaned_plus_nn_wNoNulls
# df_new["UWI (AGS)"] = df_new["UWI (AGS)_x"]
df_new["UWI"] = df_new["UWI"]
df_new["HorID"] = df_new["McMurray_Top_HorID"]
df_new["Pick"] = df_new["McMurray_Top_DEPTH"]
df_new["Quality"] = df_new["McMurray_Top_Qual"]
df_new["HorID_paleoz"] = df_new["McMurray_Base_HorID"]
df_new["Pick_paleoz"] = df_new["McMurray_Base_DEPTH"]
df_new["Quality_paleoz"] = df_new["McMurray_Base_Qual"]
df_new = df_new[["SitID","UWI","HorID","Pick","Quality","HorID_paleoz","Pick_paleoz","Quality_paleoz",'lat','lng','MM_Top_Depth_predBy_NN1thick','NN1_thickness']]
df_new

,SitID,UWI,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,lat,lng,MM_Top_Depth_predBy_NN1thick,NN1_thickness
0,102496,00/12-08-067-01W4/0,13000,475.00,3,14000,561.00,1,54.785907,-110.129320,475.00,86.00
1,102497,00/07-08-067-02W4/0,13000,515.00,3,14000,604.50,1,54.782284,-110.269446,520.50,84.00
2,102498,00/09-11-067-02W4/0,13000,480.00,3,14000,564.00,1,54.785892,-110.186851,480.00,84.00
3,102500,00/10-29-067-02W4/0,13000,549.00,3,14000,636.50,1,54.829624,-110.269422,552.50,84.00
4,102501,00/06-34-067-02W4/0,13000,529.00,2,14000,613.00,1,54.840471,-110.224832,524.00,89.00
5,102503,00/11-04-067-03W4/0,13000,488.50,2,14000,553.50,1,54.771449,-110.402983,489.00,64.50
6,102505,00/10-08-067-03W4/0,13000,501.50,2,14000,572.50,1,54.785901,-110.422131,508.00,64.50
7,102507,00/10-14-067-03W4/0,13000,553.50,2,14000,606.50,1,54.800533,-110.345762,542.00,64.50
8,102514,00/10-28-067-03W4/0,13000,493.50,3,14000,558.00,1,54.829633,-110.396621,487.00,71.00
9,102517,00/07-36-067-03W4/0,13000,536.50,2,14000,615.00,1,54.840441,-110.320301,562.00,53.00


In [20]:
print(len(df_new))

1926


In [21]:
#### Number of unique wells based on UWI
len(df_new.UWI.unique())

1926

In [22]:
df_new_test = df_new[['UWI']]

In [23]:
print(any(df_new_test.UWI == '00/11-04-067-03W4/0'))

True


## TRIAL : make dictionary simple first, no dask delayed used

In [50]:
%time
def loadAndNoFeaturesNoDask():
    count=0
    data_df=[]
    count_limit = 2000
    list_of_failed_wells = []
    ### dictionary that holds every well as key:value or "UWI":df pair
    df_w_dict ={}
    while count < count_limit:
        for file in glob.glob('../../../SPE_006_originalData/OilSandsDB/Logs/*.LAS'):
            #### NOTE: limiting wells being read-in to 101 here !!!!!!!!!!!!!!!!
            count+=1
            if count > count_limit:
                print("hit limit of count below file for loop")
                answer = [df_w_dict,list_of_failed_wells]
                return answer
            else:
                #### Load each well as a pandas dataframe using LASIO, delay compute for parallalism using dask delayed
                l_df = lasio.read(file).df()
                #l_df = l_df.compute()
                str_uwi= file[-23:-4].replace("-", "/",1)[:17]+file[-6:-4].replace("-", "/",1)
                #l_df.DEPT = l_df.DEPT.astype(float)
                ##bottom_well_depth = l_df['DEPT'].max()
                if any(df_new.UWI == str_uwi):
                    if df_new[df_new['UWI']==str_uwi]['Quality'].iloc[0] > -1:
                        l_df = l_df.reset_index()
                        print("got to UWI apppend")
                        #l_df['UWI'] = str_uwi
                        df_w_dict[str_uwi]= l_df
                        #df_w_dict[l_df['UWI'][0]]= l_df

                    else:
                        list_of_failed_wells.append(str_uwi)
                        print("could not find UWI match for the well")
                        pass
                else:
                    list_of_failed_wells.append(str_uwi)
                    pass
    answer = [df_w_dict,list_of_failed_wells]
    
    return initial_well_dict

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


In [51]:
%time initial_well_dict_NoDask = loadAndNoFeaturesNoDask()

got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI a

got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI a

got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI a

Header section Parameter regexp=~P was not found.


got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI a

got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI apppend
got to UWI a

## Results of simple function `loadAndNoFeaturesNoDask()` with no dask applied:
CPU times: user 56.8 s, sys: 708 ms, total: 57.5 s
Wall time: 58.2 s

## 2nd TRIAL : same function as above but slightly altered to use dask delayed

In [101]:
%time
def loadAndNoFeatures():
    count=0
    data_df=[]
    count_limit = 2000
    list_of_failed_wells = []
    ### dictionary that holds every well as key:value or "UWI":df pair
    df_w_dict ={}
    while count < count_limit:
        for file in glob.glob('../../../SPE_006_originalData/OilSandsDB/Logs/*.LAS'):
            #### NOTE: limiting wells being read-in to 101 here !!!!!!!!!!!!!!!!
            count+=1
            if count > count_limit:
                print("hit limit of count below file for loop")
                answer = [df_w_dict,list_of_failed_wells]
                return answer
            else:
                #### Load each well as a pandas dataframe using LASIO, delay compute for parallalism using dask delayed
                l_df = delayed(lasio.read)(file).df()
                
                str_uwi= file[-23:-4].replace("-", "/",1)[:17]+file[-6:-4].replace("-", "/",1)
                #l_df.DEPT = l_df.DEPT.astype(float)
                ##bottom_well_depth = l_df['DEPT'].max()
                if any(df_new.UWI == str_uwi):
                    if df_new[df_new['UWI']==str_uwi]['Quality'].iloc[0] > -1:
                        l_df = l_df.reset_index()
                        #print("got to UWI apppend")
                        #l_df['UWI'] = str_uwi
                        df_w_dict[str_uwi]= l_df
                        #df_w_dict[l_df['UWI'][0]]= l_df

                    else:
                        list_of_failed_wells.append(str_uwi)
                        print("could not find UWI match for the well")
                        pass
                else:
                    list_of_failed_wells.append(str_uwi)
                    pass
                
    l_df = l_df.compute()
    answer = [df_w_dict,list_of_failed_wells]
    
    return initial_well_dict

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.58 µs


In [102]:
%time initial_well_dict = loadAndNoFeatures()

hit limit of count below file for loop
CPU times: user 2.58 s, sys: 52 ms, total: 2.63 s
Wall time: 2.63 s


In [103]:
%time 
dict_of_well_df = compute(initial_well_dict)
dict_of_well_df = dict_of_well_df[0]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs


In [105]:
dict_of_well_df = dict_of_well_df[0]

In [106]:
%time 
dict_of_well_df['00/10-21-078-26W4/0']

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.54 µs


,DEPT,DELT,GR,ILD
0,411.48,310.798,59.316,22.461
1,411.73,289.773,53.073,30.512
2,411.98,252.559,48.435,34.455
3,412.23,260.422,45.294,32.269
4,412.48,287.479,40.856,27.848
5,412.73,308.897,41.623,21.463
6,412.98,299.268,47.515,17.121
7,413.23,298.019,54.158,15.136
8,413.48,301.013,60.626,12.843
9,413.73,298.447,65.089,11.869


In [107]:
type(dict_of_well_df)

dict

In [108]:
len(dict_of_well_df)

1749

## Results of simple function `loadAndNoFeatures()` with dask delayed applied:
CPU times: user 3.47 s, sys: 238 ms, total: 3.71 s
Wall time: 3.6 s

#### When using dask delayed, the speed to load wells and saves as dict of dataframes went from 58.2 s to 3.6s in terms of what %time says but this isn't quite accurate as the compute() task takes maybe 10-15 seconds not <3 seconds?????

In [109]:
### and the failed well list is:
list_of_failed_wells = initial_well_dict[1]

In [110]:
print("list_of_failed_wells",list_of_failed_wells)

list_of_failed_wells ['00/07-27-073-18W4/0', '00/10-08-067-17W4/0', '00/10-03-094-21W4/0', '00/14-03-093-19W4/0', '00/11-16-095-18W4/0', '00/07-22-093-19W4/0', '00/11-10-072-18W4/0', '00/11-24-088-22W4/0', '00/10-24-083-19W4/0', '00/15-34-073-25W4/0', '00/10-18-074-03W5/0', '00/10-17-073-17W4/0', '00/11-10-076-25W4/0', '00/10-15-074-17W4/0', 'AA/10-17-084-19W4/0', '00/03-19-081-05W5/0', '00/11-21-087-17W4/0', 'AA/06-11-089-20W4/0', '00/10-02-096-18W4/0', '00/16-01-070-20W4/0', '00/11-22-068-14W4/0', '00/14-03-077-19W4/0', '00/06-32-070-25W4/0', '00/11-08-092-21W4/0', '00/12-08-074-05W5/0', '00/10-02-099-13W4/0', '00/06-01-093-21W4/0', '00/07-06-095-21W4/0', '00/11-12-096-18W4/0', '00/11-18-088-02W5/0', '00/10-05-077-18W4/0', '00/04-26-084-15W4/0', '00/09-27-072-05W5/0', '00/12-24-080-18W4/0', '00/13-28-091-19W4/0', '00/11-34-079-18W4/0', '00/05-25-093-20W4/0', '00/04-07-091-19W4/0', '00/10-08-095-21W4/0', '00/05-09-095-16W4/0', '00/03-28-095-18W4/0', '00/08-09-070-18W4/0', '00/06-11-07

In [111]:
len(list_of_failed_wells)

251

In [112]:
#### Number of wells in dict
#### Each well is a dataframe at this point
#### No features yet!
len(dict_of_well_df)

1749

### Saving result as pickle

In [113]:
#### dumping dict of data frame to pickle file
dict_wells_df_and_Nofeatures_20180707 = dict_of_well_df
pickle.dump(dict_wells_df_and_Nofeatures_20180707, open( "dict_of__wells_df_No_features_class3_20180708a.p", "wb" ) )

------------------------------

## Next step will be to calculate features and add in features from knn notebook

-------------------------------------------------------